In [1]:
import requests
import os
import gmaps
import datetime 
import pandas as pd
import numpy as np
from scipy.stats import linregress
from matplotlib import pyplot as plt

from config import weather_key

In [ ]:

# Import cities to check pollution values
cities_to_check = pd.read_csv("Resources/100-200k_population_top_public_trans.csv")
cities_to_check

In [2]:
collected_measurements_df = pd.DataFrame()

base_url = "http://api.openweathermap.org/data/2.5/air_pollution/history?"

# December 1, 2020
startDate = pd.Timestamp('2020-12-01T12').timestamp()
# February 1, 2021
endDate = pd.Timestamp('2021-02-01T12').timestamp()

# Create series to convert data type
ser = pd.Series([startDate, endDate], dtype='int32')

# Put converted dates into variable to use
startDate = ser[0]
endDate = ser[1]

for index, row in cities_to_check.iterrows():
    
    lat = cities_to_check.loc[index, "Lat"]
    lng = cities_to_check.loc[index, "Lng"]
    city = cities_to_check.loc[index, "City"]
    state = cities_to_check.loc[index, "State"]
    zipcode = cities_to_check.loc[index, "Zipcode"]
    
    # The parameter piece of the URL
    param_url = f'lat={lat}&lon={lng}&start={startDate}&end={endDate}&appid={weather_key}'

    try:
        # Add parameters to query and convert to json
        response = requests.get(base_url + param_url)
        measurement_data = response.json()
    # Create a dataframe out of the result
        measurements_df = pd.DataFrame(measurement_data["list"])
        print(f'Checking on {city},{state}')
    
    #print and move on
    except:
        print(f'Error in JSON pull for {city},{state}')
    
    #Date and measurement conversion processing
    for index, row in measurements_df.iterrows():
    
        # Capture and convert data from UNIX time
        currentDate = measurements_df.loc[index, "dt"]
        currentDate = pd.Timestamp(currentDate, unit='s')
    
        # Capture singular AQI
        main = measurements_df.loc[index, "main"]
        aqi = main["aqi"]
    
        # Capture components breakdown
        components = measurements_df.loc[index, "components"]
        co = components["co"]
        no = components["no"]
        no2 = components["no2"]
        o3 = components["o3"]
        so2 = components["so2"]
        pm2_5 = components["pm2_5"]
        pm10 = components["pm10"]
        nh3 = components["nh3"]
    
        # Captured data to new dataframe columns
        measurements_df.loc[index, "datetime"] = currentDate
    
        measurements_df.loc[index, "co"] = co
        measurements_df.loc[index, "no"] = no
        measurements_df.loc[index, "no2"] = no2
        measurements_df.loc[index, "o3"] = o3
        measurements_df.loc[index, "so2"] = so2
        measurements_df.loc[index, "pm2_5"] = pm2_5
        measurements_df.loc[index, "pm10"] = pm10
        measurements_df.loc[index, "nh3"] = nh3
        
        measurements_df.loc[index, "aqi"] = aqi
    
    measurements_df = measurements_df.drop(columns=['main','components','dt'])
    measurements_df.loc[:,"City"] = city
    measurements_df.loc[:,"State"] = state
    measurements_df.loc[:,"Zipcode"] = zipcode
    
    collected_measurements_df = collected_measurements_df.append(measurements_df, ignore_index=True)
    
collected_measurements_df



NameError: name 'cities_to_check' is not defined